In [89]:
import requests
import pandas
from bs4 import BeautifulSoup

r = requests.get("http://www.pyclass.com/real-estate/rock-springs-wy/LCWYROCKSPRINGS/", headers={'User-agent': 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:61.0) Gecko/20100101 Firefox/61.0'})
c= r.content
soup = BeautifulSoup(c, "html.parser")
all = soup.find_all("div", {"class": "propertyRow"})

page_nr = soup.find_all("a", {"class": "Page"})[-1].text
l = []
for page in range(0, int(page_nr)*10, 10):
#     print(base_url+str(page)+ ".html")
    r = requests.get(base_url+str(page)+ ".html", headers={'User-agent': 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:61.0) Gecko/20100101 Firefox/61.0'})
    c= r.content
    soup = BeautifulSoup(c, "html.parser")
    all = soup.find_all("div", {"class": "propertyRow"})


    for item in all:
        #creating a dictionary for each property
        d = {}
        try:
            d["Locality"] = item.find_all("span", {"class", "propAddressCollapse"})[1].text
        except:
            d["Locality"] = None
        d["Address"] = item.find_all("span", {"class", "propAddressCollapse"})[0].text
        d["Price"] = item.find("h4").text.replace("\n", "").replace(" ", "")

        #number of bedrooms
        try:
            d["Bed"] = item.find("span", {"class","infoBed"}).find("b").text # extract only the number of the bedrooms
        except:
            d["Bed"] = None
        #number of areas
        try:
            d["Area"] = item.find("span", {"class","infoSqFt"}).find("b").text # extract only the number of the bedrooms
        except:
            d["Area"] = None
        #number of baths
        try:
            d["Bath"] = item.find("span", {"class","infoValuefullBath"}).find("b").text # extract only the number of the bedrooms
        except:
             d["Bath"] = None

        #number of halfbath
        try:
            d["Half Bath"] = item.find("span", {"class","infoValueHalfBath"}).find("b").text # extract only the number of the bedrooms
        except:
            d["Half Bath"] = None
        for column_group in item.find_all("div", {"class": "columnGroup"}):

            for feature_group, feature_name in zip(column_group.find_all('span', {"class":"featureGroup"}),
                                                  column_group.find_all("span", {"class":"featureName"})):
    #            print(feature_group.text, feature_name.text)
                if "Lot Size" in feature_group.text:
                    d["Lot Size"] = feature_name.text            
        l.append(d)                                               
df = pandas.DataFrame(l)
df.to_csv("Output.csv")
